In [11]:
# Install the Prediction Guard client
! pip install -U predictionguard

# Python Imports
import os
import json

import pandas as pd
from getpass import getpass

# Load the transcripts from the csv file named "cleaned_output_file"
transcripts = pd.read_csv('cleaned_output_file.csv')

# Convert the transcripts dataframe to a dictionary with id as the key and text as the value
transcripts = transcripts.set_index('id')['text'].to_dict()

# Load the test data from the csv file named "test.csv"
test = pd.read_csv('testpg.csv')

pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

import predictionguard as pg

# Create a Prediction Guard client object
pg = pg.Client(
    model="Nous-Hermes-Llama2-13B",
    output_type="json",
    output_structure={
        "answer": "string",
        "question_id": "string"
    }
)

# Initialize an empty list to store the answers
answers = []

# Loop over all the rows in the test data
for _, row in test.iterrows():
    # Get the question and the transcript id from the row
    question = row['Question']
    transcript_id = row['Transcript']

    # Get the transcript text from the transcripts dictionary
    transcript = transcripts[transcript_id]

    # Create an LLM "prompt" that we will use to extract certain
    # information from the transcript.
    prompt = f"""### Instruction:
    Extract the patient's condition from the below input.

    ### Input:
    {transcript}

    ### Response:""".format(transcript=transcript)

    # Use the Prediction Guard instance to generate an answer based on the prompt
    # Add a try-except block to catch and handle any errors or exceptions
    try:
        answer = pg.generate(prompt=prompt)
    except Exception as e:
        # Print the error message and the question id to the console
        print(f"Error: {e} for question id {row['Id']}")
        # Set the answer to None
        answer = None

    # Use the Prediction Guard instance to validate the answer and check if it matches the output structure
    # Add a try-except block to catch and handle any errors or exceptions
    try:
        valid = pg.validate(answer)
    except Exception as e:
        # Print the error message and the question id to the console
        print(f"Error: {e} for question id {row['Id']}")
        # Set the valid flag to False
        valid = False

    # Append the answer and the question id to the answers list if the answer is valid
    if answer is not None and valid:
        answers.append([row['Id'], answer['answer']])
    else:
        answers.append([row['Id'], ""])


Enter your Prediction Guard access token: ········


TypeError: Client.__init__() got an unexpected keyword argument 'model'

In [ ]:
# Convert the answers list to a pandas dataframe
answers = pd.DataFrame(answers, columns=["Id", "Text"])
answers.to_csv("pg_submission_11nov_up1.csv", index=False)  # Removed index and index_label